In [167]:
import tushare as ts
import os
import datetime
import pymongo
import pandas
import copy

def ConnectDB():
    client = pymongo.MongoClient("127.0.0.1", 27017)
    return client

def Today():
    today = datetime.datetime.now().replace(hour=0, minute=0, second=0, microsecond=0)
    return today

def SaveResult(client, code, date, strategy, data):
    db = client.Result
    collection = db.Recommend
    collection.replace_one({
        "code": code,
        "date": date,
        "strategy": strategy
    }, data, upsert=True)
    

In [171]:
def Strategy_1():
    client = ConnectDB()
    
    startDate = datetime.datetime.now() + datetime.timedelta(days=-60)
    stocks = client.Common.Stock.find({"foundTime": {"$lt": startDate}})
    threshold = 9.9
    today = Today()
    found = []
    minDays = 1000

    for item in stocks:
        code = item["code"]
        collection = client.Trade[code]
        res = list(collection.find({}, limit=4, sort=[("date", pymongo.DESCENDING)]))
        if len(res) < 4:
            continue
        changes = []
        for i in range(3):
            if res[i+1]["close"] == 0:
                d = 0
            else:
                d = (res[i]["close"] / res[i+1]["close"] - 1) * 100
            changes.append(d)
        if changes[0] < threshold or (changes[1] <= threshold and changes[2] <= threshold):
            continue
        days = (today - res[3]["date"]).days
        if days > minDays:
            continue
        if days < minDays:
            minDays = days
            found = []
        found.append({
            "code": code,
            "name": item["name"],
            "changes": changes
        })
        data = {
            "code": code,
            "name": item["name"],
            "date": today,
            "strategy": 1,
            "changes": changes
        }
        SaveResult(client, code, today, 1, data)

    if len(found) > 0:
        print pandas.DataFrame(found, columns=["code", "name", "changes"])
    else:
        print("No Result")
        
    client.close()

In [173]:
def Strategy_3():
    client = ConnectDB()
    
    stocks = client.Common.Stock.find({})
    today = Today()
    found = []
    minDays = 1000

    for item in stocks:
        code = item["code"]
        collection = client.Trade[code]
        res = list(collection.find({}, limit=10, sort=[("date", pymongo.DESCENDING)]))
        if len(res) < 10:
            continue
        days = (today - res[-1]["date"]).days
        if days > minDays:
            continue
        if res[0]["close"] < res[1]["close"]:
            continue
        if days < minDays:
            minDays = days
            found = []
        v10 = 0
        volumeDays = 2
        vv = 0
        p5 = 0
        for i in range(10):
            v10 = v10 + res[i]["volume"]
            if i < volumeDays:
                vv = vv + res[i]["volume"]
            if i < 5:
                p5 = p5 + res[i]["close"]
        v10 = int(v10 / 10)
        vv = int(vv / volumeDays)
        p5 = int(p5 / 5)
        close = res[0]["close"]
        if vv >= v10 * 2 and close >= p5:
            item = {
                "code": code,
                "name": item["name"],
                "volume_" + str(volumeDays): vv,
                "volume_10": v10,
                "price_today": close,
                "price_5": p5,
                "factor": (1.0 * vv / v10),
            }
            found.append(item)

        found = sorted(found, key=lambda item: item["factor"], reverse=True)

    found = found[:10]
    for item in found:
        data = copy.copy(item)
        data["date"] = today
        data["strategy"] = 3
        SaveResult(client, item["code"], today, 3, data)
        
    if len(found) > 0:
        print pandas.DataFrame(found[:10])
    else:
        print("No Result")
        
    client.close()

In [179]:
def Strategy_4():
    client = ConnectDB()
    
    today = Today()
    stocks = client.Common.Stock.find({
        "$and": [
            {"value": {"$exists": True}},
            {"value": {"$ne": 0}}
        ]})

    found = []

    for item in stocks:
        code = item["code"]
        collection = client.Trade[code]
        res = collection.find_one({"date": today})
        if not res:
            continue
        if "last_close" not in res:
            continue
        if res["close"] <= res["last_close"]:
            continue
        value = res["tradeValue"] * 10000
        if "amount" in res:
            amount = res["amount"]
        else:
            amount = res["volume"] * (res["open"] + res["close"]) / 2 * 100
        item = {
            "code": code,
            "name": item["name"],
            "value": value / 1e8,
            "amount": amount / 1e8,
            "factor": amount / value
        }
        found.append(item)
        
    found = sorted(found, key=lambda item: item["factor"], reverse=True)[:10]
    for item in found:
        data = copy.copy(item)
        data["date"] = today
        data["strategy"] = 4
        SaveResult(client, item["code"], today, 4, data)
        
    print(pandas.DataFrame(found[:10], columns=["code", "name", "factor", "amount", "value"]))

    client.close()

In [223]:
def Strategy_2():
    client = ConnectDB()
    
    today = Today()
    conceptDict = {}
    Threshold = 9.9
    
    for item in client.Common.Stock.find({}):
        code = item["code"]
        collection = client.Trade[code]
        res = collection.find_one({"date": today})
        if not res:
            continue
        if "last_close" not in res:
            continue
        change = (res["close"] / res["last_close"] - 1) * 100
        if change < Threshold:
            continue
    
        for concept in item["concept"]:
            if concept == u"次新股":
                continue
            if concept not in conceptDict:
                conceptDict[concept] = []
            data = {
                "code": item["code"],
                "name": item["name"],
                "change": change
            }
            conceptDict[concept].append(data)
    
    concepts = []
    for k, v in conceptDict.iteritems():
        if len(v) < 3:
            continue
        data = {
            "concept": k,
            "stocks": v,
            "count": len(v),
            "codes": []
        }
        for s in v:
            data["codes"].append(s["code"])
        concepts.append(data)
    
    found = sorted(concepts, key=lambda item: item["count"], reverse=True)
    
    print(pandas.DataFrame(found, columns=["concept", "count", "codes"]))
    
    stockDict = {}
    for k in found:
        for s in k["stocks"]:
            code = s["code"]
            if code not in stockDict:
                stockDict[code] = {
                    "code": code,
                    "name": s["name"],
                    "change": s["change"],
                    "concepts": [],
                    "date": today,
                    "strategy": 2,
                }
            stockDict[code]["concepts"].append(k["concept"])
    for code, v in stockDict.iteritems():
        SaveResult(client, code, today, 2, v)
        
    client.close()

In [227]:
def FilterSameStrategy():
    client = ConnectDB()
    today = Today()
    
    stocks = {}
    for s in client.Result.Recommend.find({"date": today}):
        code = s["code"]
        if code not in stocks:
            stocks[code] = {
                "code": code,
                "name": s["name"],
                "strategies": []
            }
        stocks[code]["strategies"].append(s["strategy"])
        
    result = []
    for k, v in stocks.iteritems():
        if len(v["strategies"]) > 1:
            result.append({
                "code": k,
                "name": v["name"],
                "strategies": v["strategies"]
            })
    print pandas.DataFrame(result)

In [263]:
print("Strategy 1")
Strategy_1()
print("\nStrategy 2")
Strategy_2()
print("\nStrategy 3")
Strategy_3()
print("\nStrategy 4")
Strategy_4()
print("\nSummary")
FilterSameStrategy()

Strategy 1
     code  name                                        changes
0  600476  湘邮科技  [9.98838559814, 10.0319488818, 3.43688037013]
1  002524  光正集团                   [10.0, 10.0, -1.80032733224]
2  300526  中潜股份  [9.98751560549, -6.20608899297, 9.9806825499]

Strategy 2
  concept  count                     codes
0    阿里概念      3  [300251, 600476, 600332]

Strategy 3
     code    factor  name  price_5  price_today  volume_10  volume_2
0  603301  4.519107  振德医疗       66        74.48      21223     95909
1  000637  3.138058  茂化实华        5         5.39      52869    165906
2  000678  2.939347  襄阳轴承        6         6.79      45719    134384
3  002755  2.673889  东方新星       25        26.72       7292     19498
4  002458  2.586974  益生股份       13        15.10      77414    200268
5  300370  2.450077  安控科技        3         4.14     189801    465027
6  000650  2.345354  仁和药业        5         5.46     145636    341568
7  002691  2.304654  冀凯股份       26        29.40      28360     65360
8  603

In [268]:
today = Today()
minDate = today + datetime.timedelta(days=-10)
collection = client.Result.Recommend
cursor = collection.find({"date": {"$gt": minDate}}) \
    .sort("date", pymongo.DESCENDING)
    
stocks = {}
daysDiff = -1
daysCount = 0
daysNeed = 4
startDate = None
for s in cursor:
    diff = (today - s["date"]).days
    if diff > daysDiff:
        daysDiff = diff
        daysCount = daysCount + 1
    if daysCount > daysNeed:
        break
    if daysCount < daysNeed:
        continue
    
    startDate = s["date"]
    code = s["code"]
    if code not in stocks:
        stocks[code] = {
            "code": code,
            "name": s["name"],
            "strategies": [s["strategy"]]
        }
    else:
        stocks[code]["strategies"].append(s["strategy"])
        
found = []
for code, stock in stocks.iteritems():
    res = []
    for s in client.Trade[code].find({"date": {"$gt": startDate}}):
        res.append(s)
    if len(res) < daysNeed - 1:
        continue
    buyPrice = res[0]["open"]
    sellPrice1 = res[1]["high"]
    sellPrice2 = res[2]["high"]
    if buyPrice == 0:
        continue
    change1 = (sellPrice1 / buyPrice - 1) * 100
    change2 = (sellPrice2 / buyPrice - 1) * 100
    found.append({
        "code": code,
        "name": stock["name"],
        "strategies": stock["strategies"],
        "change1": change1,
        "change2": change2,
        "buy": buyPrice,
        "sell": sellPrice
    })
    
found = sorted(found, key=lambda item: item["change2"], reverse=True)
print pandas.DataFrame(found, columns=["code", "name", "change1", "change2", "buy", "sell", "strategies"])

      code   name     change1     change2     buy   sell strategies
0   300667   必创科技    5.604720   20.176991   67.80  20.49        [1]
1   002931   锋龙股份   17.021277   18.872340   47.00  20.49        [3]
2   300624   万兴科技    6.956522   18.747826  115.00  20.49        [4]
3   002930   宏川智慧   12.426036   17.554241   25.35  20.49        [4]
4   300634   彩讯股份   17.836101   16.871987   56.01  20.49        [4]
5   601518   吉林高速   16.032609   16.304348    3.68  20.49     [1, 3]
6   002864   盘龙药业    9.562080   15.105790   60.97  20.49        [4]
7   300718   长盛轴承    6.184866   14.589941   44.14  20.49        [1]
8   300366   创意信息    6.103994   13.564431   13.27  20.49     [1, 2]
9   300504   天邑股份    2.422222   13.177778   45.00  20.49        [4]
10  002856   美芝股份   10.367545   12.239945   28.84  20.49        [4]
11  300730   科创信息   10.675431   10.788758   44.12  20.49        [4]
12  002908   德生科技    5.555556    9.388889   36.00  20.49        [4]
13  600695   绿庭投资    7.591623    7.591623    7.6